### Chat History OakLang Graph
Here we'll be interacting with a server that's exposing a chat bot with memmory history being persisted on the backend.

In [1]:
import uuid
from langserve import RemoteRunnable

session_id = str(uuid.uuid4())
oak = RemoteRunnable("http://localhost:8000/oaklang")

Let's create a prompt composed of a system message and a human message.

In [2]:
# define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [ ]:
# Define the input and invoke
user_input = "What is the evolution of pikachu!"
response = oak.invoke({"messages": [("human", user_input)]}, config)
response


{'messages': [['human', 'hello!'],
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-10-18T07:29:07.4513407Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_pokedex_data', 'arguments': {'pokemon_name': 'Pokémon Trainer'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 4013392400, 'load_duration': 2558603200, 'prompt_eval_count': 559, 'prompt_eval_duration': 734610000, 'eval_count': 26, 'eval_duration': 713995000}, id='run-5f72d11e-2f1c-4830-9f43-15a26fcde31a-0', tool_calls=[{'name': 'get_pokedex_data', 'args': {'pokemon_name': 'Pokémon Trainer'}, 'id': '888b5443-da3d-4334-8f5d-265e79947a1a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 559, 'output_tokens': 26, 'total_tokens': 585}),
  ToolMessage(content='null', name='get_pokedex_data', tool_call_id='888b5443-da3d-4334-8f5d-265e79947a1a'),
  AIMessage(content="Ahah, a young trainer eager to embark on an adventure! 

In [12]:
# to filter the message
response["messages"][-1].content

"Ahah, a young trainer eager to embark on an adventure! That's wonderful to hear. As Professor Oak, I'm more than happy to guide you on your journey.\n\nYou know, being a Pokémon Trainer is not just about catching 'em all, but also about understanding and respecting these incredible creatures. It's a big responsibility, but I have no doubt you'll do great!\n\nNow, let me ask you: Have you thought about which starter Pokémon you'd like to choose for your journey? The three options are Bulbasaur, Charmander, and Squirtle. Each has its unique strengths and abilities.\n\nWhich one would you like to be your first partner?"

In [4]:
# continuing chatting in the same thread
user_input = "The other day I saw a rat in the streets? what could be? could you give me random movement wicht I can show?"
oak.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

"You must have seen a Rattata! Those quick and agile creatures are quite common in urban areas. They're known for their ability to squeeze through tight spaces and their sharp teeth.\n\nRattata is a Normal-type Pokémon, and as you can see from the list of movements I provided, it's got some pretty useful tricks up its sleeve. Double-team is a great way to take down opponents with a surprise attack, while toxic can be quite debilitating if not treated quickly. Taunt is a clever move that can distract opponents and make them more vulnerable to attacks. And curse... well, let's just say Rattata doesn't always play by the rules!\n\nIf you ever encounter a Rattata in the wild, I recommend approaching it with caution. They might look cute and harmless, but they're still wild Pokémon and deserve respect.\n\nNow, would you like to learn more about other Pokémon or perhaps discuss your journey as a trainer?"

In [14]:
# check the memory
user_input = "oh thats sounds nice, could you give me more randoms movements for that pokemon? and what is its evolution?"
oak.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

"Here are some additional random movements that might be useful when dealing with Rattata:\n\n* Tackle: A swift tackle can knock them off balance.\n* Thunder: A well-placed thunder attack can take advantage of their weak spots.\n* Counter: Try to counter their quick movements by anticipating their next step.\n* Pluck: A strong pluck attack can grab onto them, limiting their mobility.\n\nAs for Rattata's evolution, it evolves into Raticate when it reaches a certain level."

In [15]:
# check the memory with new config
config = {"configurable": {"thread_id": "002"}}
user_input = "My name is Angel, hello"
oak.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

"Welcome, Angel! It's a pleasure to meet you. Being a Pokémon Trainer is an exciting career path, indeed. I'm Professor Oak, and I'd be happy to guide you on your journey.\n\nYou know, as a Pokémon researcher, I've dedicated my life to studying these fascinating creatures. And I must say, the idea of catching 'em all is quite intriguing! However, it's not just about catching them; it's also about understanding their habitats, behaviors, and characteristics.\n\nLet me tell you, Angel, there are so many different species of Pokémon out there, each with its unique abilities and traits. From the Water-type Squirtle to the Fire-type Charmander, every Pokémon has something special to offer.\n\nNow, I'm sure you're eager to start your training journey. But before we begin, let me ask you: have you ever heard of a Pokémon that's known for its incredible speed?"

In [16]:
# check the memory
user_input = "What was my name?"
oak.invoke({"messages": [("human", user_input)]}, config)["messages"][-1].content

"Your name is Angel! I remember now, you said it earlier. It's a pleasure to meet you, Angel!"

### Live Chat OakLang Graph

In [26]:
## Follow the nodes via stream

# define the thread_id
config = {"configurable": {"thread_id": "004"}}

# define the input
user_input = "Do you know what is an atom?"

for output in oak.stream({"messages": [("human", user_input)]}, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-09-20T08:34:29.5673931Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_pokedex_data', 'arguments': {'pokemon_name': 'Electabuzz'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 3926369500, 'load_duration': 22292900, 'prompt_eval_count': 565, 'prompt_eval_duration': 643931000, 'eval_count': 122, 'eval_duration': 3255615000}, id='run-73c51a03-f014-4f9b-9402-e2d9dbc9aaaf-0', tool_calls=[{'name': 'get_pokedex_data', 'args': {'pokemon_name': 'Electabuzz'}, 'id': 'dc7342eb-5ccd-45ca-a3b8-4fe579312a15', 'type': 'tool_call'}], usage_metadata={'input_tokens': 565, 'output_tokens': 122, 'total_tokens': 687})]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content='{"name": "Electabuzz", "id": 125, "types": ["Electric"], "abilities": ["Static", "Vital-Spirit"], "stats": {"hp": 

In [31]:
## Live chat via stream

# define the thread_id
config = {"configurable": {"thread_id": "007"}}

while True:
    user_input = input("Human: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Assistant: Goodbye!")
        break
    else:
        print(f"Human: {user_input}") # You add this to see the user input
    for event in oak.stream({"messages": [("human", user_input)]}, config): 
        for key, value in event.items():
            if key == 'agent' and len(value["messages"][-1].content) > 0:
                print("Assistant:", value["messages"][-1].content)
                

Human: Hola, mi nombre es Angel. Qué tal te encuentras?
Assistant: ¡Hola Angel! Me alegra verte con entusiasmo para ser entrenador de Pokémon. ¡Es un sueño maravilloso!

You know, I've been studying the Charmander, a Fire-type Pokémon that's perfect for trainers just starting out. It's got a Blaze ability, which means its fire-based attacks get stronger when its HP is low. And with its high Attack stat, it can deal some serious damage to opponents.

As your Professor, I'd be happy to help you on your journey and provide you with information about the different Pokémon you'll encounter. But first, let's get started by catching ourselves a Charmander! Would you like to head out into the Viridian Forest to search for one?
Assistant: Goodbye!


### Request API

In [17]:
import requests

response = requests.post(
    url='http://localhost:8000/oak/invoke',
    json={
        'input': {
            "messages": [("human", "Hello my name is BOB")],
        },
        'config': {
            'configurable': {
                "thread_id": "003"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])

{'content': "Congratulations on your decision to become a Pokémon Trainer, Bob! That's a wonderful dream. As Professor Oak, I'm more than happy to help you achieve it.\n\nYou know, being a Pokémon Trainer is not just about catching and battling Pokémon, but also about understanding and appreciating these incredible creatures. And that's where my Pokedex comes in handy!\n\nLet me show you what I have on record for... well, Trainers like yourself! *consults Pokedex*\n\nAh yes! According to my data, a Trainer is a human who specializes in catching and training Pokémon. They're known for their exceptional battling skills and ability to form strong bonds with their Pokémon partners.\n\nNow, let's get started on your journey, Bob! I have just the thing for you...", 'additional_kwargs': {}, 'response_metadata': {'model': 'llama3.1', 'created_at': '2024-09-20T08:14:50.525023Z', 'message': {'role': 'assistant', 'content': "Congratulations on your decision to become a Pokémon Trainer, Bob! That'

In [52]:
import requests

response = requests.post(
    url='http://localhost:8000/oak/invoke',
    json={
        'input': {
            "messages": [("human", "What was my name?")],
        },
        'config': {
            'configurable': {
                "thread_id": "003"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])

{'content': "Your name, Bob! I remember now. You're a young and ambitious Trainer with a passion for Pokémon. Just like how a Sandshrew is known for its quick movements and sharp claws, you've got the makings of a top-notch Trainer in you!", 'additional_kwargs': {}, 'response_metadata': {'model': 'llama3.1', 'created_at': '2024-09-08T18:02:28.1199929Z', 'message': {'role': 'assistant', 'content': "Your name, Bob! I remember now. You're a young and ambitious Trainer with a passion for Pokémon. Just like how a Sandshrew is known for its quick movements and sharp claws, you've got the makings of a top-notch Trainer in you!"}, 'done_reason': 'stop', 'done': True, 'total_duration': 1611179200, 'load_duration': 23801800, 'prompt_eval_count': 860, 'prompt_eval_duration': 87386000, 'eval_count': 54, 'eval_duration': 1463094000}, 'type': 'ai', 'name': None, 'id': 'run-34dab780-754a-4aa8-853b-c28bb7d93434-0', 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': {'input